In [4]:
import pandas as pd

pd.set_option("display.max_rows", 5)

# Data Analysis guide

> ⚠️ This guide assumes that you have some familiarity with pandas.

In [5]:
from siuba.data import cars

cars

,cyl,mpg,hp
0,6,21.0,110
1,6,21.0,110
...,...,...,...
30,8,15.0,335
31,4,21.4,109


## Overview

### Split-apply-combine

In [8]:
from siuba import _


,cyl,mpg,hp,hp2
0,6,21.0,110,111
1,6,21.0,110,111
...,...,...,...,...
30,8,15.0,335,336
31,4,21.4,109,110


## Split with group_by()

## Apply with `_`

## Combine with verbs

## Connecting back to pandas

## Debugging